In [2]:
import pandas as pd 
import numpy as np

# DT 문제 1번

#### ※함수에 들어가 있는 변수나 flow는 본인이 바꾸셔도 가능하며 결과만 똑같이 나오면 됩니다!
#### ※hard코딩(이 데이터셋에만 적용되는 코딩방법) 말고 전체 데이터에 적용 가능하게 함수를 짜주셔야 합니다.

# Data Loading

In [4]:
pd_data = pd.read_csv('https://raw.githubusercontent.com/AugustLONG/ML01/master/01decisiontree/AllElectronics.csv')
pd_data.drop("RID",axis=1, inplace = True) #RID는 그냥 순서라서 삭제
pd_data

,age,income,student,credit_rating,class_buys_computer
0,youth,high,no,fair,no
1,youth,high,no,excellent,no
2,middle_aged,high,no,fair,yes
3,senior,medium,no,fair,yes
4,senior,low,yes,fair,yes
5,senior,low,yes,excellent,no
6,middle_aged,low,yes,excellent,yes
7,youth,medium,no,fair,no
8,youth,low,yes,fair,yes
9,senior,medium,yes,fair,yes


# 1. Gini 계수에 대한 함수 만들기.

- Input은 Dataframe과 label이름으로 만들어주세요!
- 해당 결과는 아래와 같이 나와야 합니다!

In [1]:
def get_gini(df, label):
    aa = df[label].value_counts()  #label 분포 
    a_class = aa.index   #label 클래스 
    a_counts = aa.values #클래스 각각의 개수 
    mother = a_counts.sum()  #전체 개수
    
    gini = 1
    for value in a_counts:  #gini계수 공식
        gini -= (value/mother)**2
    return gini    

In [5]:
get_gini(pd_data, "age")

0.663265306122449

In [11]:
get_gini(pd_data,'class_buys_computer')

0.4591836734693877

# 2. Feature의 Class를 이진 분류로 만들기
## ex) {A,B,C} -> ({A}, {B,C}), ({B}, {A,C}), ({C}, {A,B})
- Powerset은 모든 조합을 뱉어내게 됩니다. 이건 그냥 완성된걸 드릴게요.

In [7]:
def get_power_set(s):
    power_set=[[]]
    for elem in s:  #하위 집합 루프
        for sub_set in power_set:  #나머지 하위집합 포함하는 새로운 집합 추가
              power_set=power_set+[list(sub_set)+[elem]]   
    return power_set

In [8]:
get_power_set(pd_data.age.unique())

[[],
 ['youth'],
 ['middle_aged'],
 ['youth', 'middle_aged'],
 ['senior'],
 ['youth', 'senior'],
 ['middle_aged', 'senior'],
 ['youth', 'middle_aged', 'senior']]

- 저 위에 부분 집합 중 우리가 원하는 집합(이진 분류)만 골라 내야하겠죠?
- 그 함수를 get_binary_split로 완성해주세요!
- 완성된 내용은 다음과 같이 나와야 합니다.

In [9]:
def get_binary_split(df, attribute):
    result = []
    cl = get_power_set(df[attribute].unique())  #feature들에 대해서 부분집합 만들기
    cl.pop()  #처음과 끝 집합(공집합, 전체집합) 삭제
    cl.pop(0)
    result = cl
    return result

In [10]:
get_binary_split(pd_data,"age")

[['youth'],
 ['middle_aged'],
 ['youth', 'middle_aged'],
 ['senior'],
 ['youth', 'senior'],
 ['middle_aged', 'senior']]

# 3. 다음은 모든 이진분류의 경우의 Gini index를 구하는 함수 만들기
- 위에서 완성한 함수를 사용하여 만들어주세요!
- DataFrame의 index를 사용하여 만들면 굉장히 편합니다..! 예시를 아래에서 보여드릴게요.
- 결과는 아래 아래 아래 줄과 같아야 합니다.

In [12]:
A = set([1,2,3])
B = set([3,4,5])
A.union(B) #A와 B의 합집합
pd_data.loc[A.union(B)]

,age,income,student,credit_rating,class_buys_computer
1,youth,high,no,excellent,no
2,middle_aged,high,no,fair,yes
3,senior,medium,no,fair,yes
4,senior,low,yes,fair,yes
5,senior,low,yes,excellent,no


In [73]:
def get_attribute_gini_index(df, attribute, label):
    result = {}
    binarysplit = get_binary_split(df, attribute) #이진분류
    gini_1 = gini_2 = None
    for i in range(len(binarysplit)):
        if len(binarysplit[i]) == 1 :  #분류한 집합 중 원소가 하나 있을 경우 ex) 4 -> (2,2) (1,3) 에서 1
            gini_1 = get_gini(df[df[attribute] == str(binarysplit[i][0])], label)  #분류한 집합 원소에 해당하는 데이터 gini계수 구하기
            gini_2 = get_gini(df[df[attribute] != str(binarysplit[i][0])], label)  #나머지 데이터 gini 계수 구하기
            nrow = df[df[attribute] == binarysplit[i][0]].shape[0]  #아래 result를 구할 때 사용하기 위해 나뉜 후 데이터 개수 저장! 
        elif len(binarysplit[i]) > 1:  #분류한 집합 중 원소가 하나 이상 있을 경우 ex) 4 -> (2,2) (1,3) 에서 2나 3
            a = binarysplit[i]
            d1 = pd.DataFrame()
            for k,j in enumerate(a):  #집합 원소들을 클래스로 가지는 데이터 분류 (원소가 2개 이상이기 때문에 데이터 아래로 붙히기)
                d = df[df[attribute] == j]
                d1 = pd.concat([d1,d])
            d2 = pd_data.drop(d1.index)  #나머지 데이터
            gini_1 = get_gini(d1, label)  #위에서 만든 데이터로 gini계수 구하기
            gini_2 = get_gini(d2, label)  
            nrow = d1.shape[0]   #역시 result를 구할 때 사용하기 위해 나뉜 후 데이터 개수 저장!
            binarysplit[i] = (binarysplit[i][0] + "_" + binarysplit[i][1]).split()  #원소가 2개 이상('youth','senior')일 때 "youth_senior"처럼 한 단어로 이름 변경
        result[binarysplit[i][0]] = (nrow/df.shape[0])*gini_1 + (1-(nrow/df.shape[0]))*gini_2  #최종 gini계수 dictionary형태로 저장 
    return result

In [74]:
get_attribute_gini_index(pd_data, "age", "class_buys_computer")

{'youth': 0.3936507936507936,
 'middle_aged': 0.35714285714285715,
 'youth_middle_aged': 0.4571428571428572,
 'senior': 0.45714285714285713,
 'youth_senior': 0.35714285714285715,
 'middle_aged_senior': 0.39365079365079364}

- 여기서 가장 작은값으로 분류를 해야겠죠?

In [48]:
min(get_attribute_gini_index(pd_data, "age", "class_buys_computer").items())

('middle_aged', 0.35714285714285715)

# 다음의 문제를 위에서 작성한 함수를 통해 구한 값으로 보여주세요!
## 문제1) income의 이진분류를 얻는 함수 get_binary_split(pd_data, "income")을 통해 보여주세요.

## 문제2) 가장 Gini계수가 높은 Feature 즉 분류를 하는데 가장 중요한 변수를 선정하시고 get_attribute_gini_index함수를 통해 Gini index를 제시해주세요.

## 문제3) 2에서 구한 Feature로 DataFrame을 분류 해주시고 나눠진 2개의 클래스에서 각각 다음으로 중요한 Feature를 선정해주시고 Gini index를 제시해주세요.

In [49]:
##문제1
get_binary_split(pd_data, "income")

[['high'],
 ['medium'],
 ['high', 'medium'],
 ['low'],
 ['high', 'low'],
 ['medium', 'low']]

In [58]:
##문제2
ginis = {'age':min(get_attribute_gini_index(pd_data, "age", "class_buys_computer").values()),  #분류하는 데 가장 중요한 변수 선정
         "income":min(get_attribute_gini_index(pd_data, "income", "class_buys_computer").values()),
         'student': min(get_attribute_gini_index(pd_data, "student", "class_buys_computer").values()),
        "crdit_rating": min(get_attribute_gini_index(pd_data, "credit_rating", "class_buys_computer").values())}
print(min(ginis),ginis[min(ginis)])

age 0.35714285714285715


In [70]:
##문제3
data1 = pd_data[pd_data["age"]=="middle_aged"]  #데이터 분류
data2 = pd_data[pd_data["age"]!="middle_aged"]

ginis1 = {"income":min(get_attribute_gini_index(data1, "income", "class_buys_computer").values()),  #가장 작은 gini계수와 해당 feature 찾기
         'student': min(get_attribute_gini_index(data1, "student", "class_buys_computer").values()),
        "credit_rating": min(get_attribute_gini_index(data1, "credit_rating", "class_buys_computer").values())}
ginis2 = {'age':min(get_attribute_gini_index(data2, "age", "class_buys_computer").values()),
         "income":min(get_attribute_gini_index(data2, "income", "class_buys_computer").values()),
         'student': min(get_attribute_gini_index(data2, "student", "class_buys_computer").values()),
        "credit_rating": min(get_attribute_gini_index(data2, "credit_rating", "class_buys_computer").values())}

print(min(ginis1),ginis[min(ginis1)])
print(min(ginis2),ginis[min(ginis2)])

crdit_rating 0.42857142857142855
age 0.35714285714285715
